In [ ]:
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from PIL import Image
import torchvision.models as models


import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
from torch.autograd import Variable

import shutil

import copy
import os
import numpy as np

import itertools
import torch
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

import datetime
from time import time

import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset

import wandb

if not torch.cuda.is_available():
    raise RuntimeError("You should enable GPU runtime!!")
device = torch.device("cuda")
torch.set_grad_enabled(True)
%matplotlib inline

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import f1_score
import seaborn as sn
import itertools

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

In [ ]:
# We use wandb to log our results
wandb.login()

# 1. Data preprocesing
Read the execution parameters. Construct the dataloader and apply the transformations. We will use an impution of NaNs of Knn

In [ ]:
first_run = False
second_run = False
num_run = 2004
epochs = 50
learning_rate= 1e-6
start = 82272
N_images = 82273
torch.backends.cudnn.benchmark = True

In [ ]:
# If there is GPU, then use GPU.
import torch
if(torch.cuda.is_available()):
    print("SI")
else:
    print("NO")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
image_data = pd.read_csv("C:/Users/48057074t/Desktop/Postgrau/prejecte/FonsUll/data_imputed_KNN_5.csv", sep = ';', encoding="latin-1")

In [ ]:
class Diabetic_Retinopathy_Dataset(Dataset): 
  def __init__(self, transform=None):

    self.image_data = pd.read_csv('FonsUll/data_imputed_KNN_5.csv', sep = ',', encoding="iso-8859-1")
    self.image_data.columns = ["ind", "ID_Image", "ID_Pacient", "Eye", "Retinopathy", "Age", "Sex", "HbA1c", "Total Cholesterol", "LDL", "HDL", "Systolic Arterial Pressure", "Diastolic Arterial Pressure"] #, "Cardiovascular Risk Score"]
    self.target_col = "Retinopathy"

    # METADATA

    self.image_data = self.image_data.drop("ID_Pacient", axis = 1)
    self.image_data = self.image_data.drop("Eye", axis = 1)
    self.image_data = self.image_data.drop("ind", axis = 1)
    
    self.image_data["Retinopathy"] = self.image_data["Retinopathy"].astype(int)

    self.image_data["Age"] = self.image_data["Age"].astype(np.float32)
    self.image_data["Age"] = (self.image_data["Age"] -self.image_data["Age"].mean())/self.image_data["Age"].std()

    self.image_data["Sex"] = self.image_data["Sex"].apply(lambda x: 1 if x == 'D' else 0).astype(np.float32)

    self.image_data["HbA1c"] = self.image_data["HbA1c"].astype(np.float32)
    self.image_data["HbA1c"] = (self.image_data["HbA1c"] -self.image_data["HbA1c"].mean())/self.image_data["HbA1c"].std()

    self.image_data["Total Cholesterol"] = self.image_data["Total Cholesterol"].astype(np.float32)
    self.image_data["Total Cholesterol"] = (self.image_data["Total Cholesterol"] -self.image_data["Total Cholesterol"].mean())/self.image_data["Total Cholesterol"].std()

    self.image_data["LDL"] = self.image_data["LDL"].astype(np.float32)
    self.image_data["LDL"] = (self.image_data["LDL"] -self.image_data["LDL"].mean())/self.image_data["LDL"].std()

    self.image_data["HDL"] = self.image_data["HDL"].astype(np.float32)
    self.image_data["HDL"] = (self.image_data["HDL"] -self.image_data["HDL"].mean())/self.image_data["HDL"].std()
    
    self.image_data["Systolic Arterial Pressure"] = self.image_data["Systolic Arterial Pressure"].astype(np.float32)
    self.image_data["Systolic Arterial Pressure"] = (self.image_data["Systolic Arterial Pressure"] -self.image_data["Systolic Arterial Pressure"].mean())/self.image_data["Systolic Arterial Pressure"].std()

    self.image_data["Diastolic Arterial Pressure"] = self.image_data["Diastolic Arterial Pressure"].astype(np.float32)
    self.image_data["Diastolic Arterial Pressure"] = (self.image_data["Diastolic Arterial Pressure"] -self.image_data["Diastolic Arterial Pressure"].mean())/self.image_data["Diastolic Arterial Pressure"].std()

    self.ID_Image = self.image_data["ID_Image"]
    
    self.y = self.image_data[self.target_col].astype(np.float32)

    # Image
    self.transform = transform 

  def __getitem__(self, i):
    ID_imatge, label = self.image_data.loc[i,['ID_Image','Retinopathy']]
    path = f"C:/Users/48057074t/Desktop/Postgrau/prejecte/dataset_processat_full/classe_{label}/{ID_imatge}"
    image = cv2.imdecode(np.fromfile(path, dtype=np.uint8), cv2.IMREAD_UNCHANGED)
    cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = self.transform(image=image)["image"]
    
    if isinstance(i, torch.Tensor):
        i = i.tolist()

    return image,  torch.tensor((self.image_data.drop("ID_Image", axis = 1).drop(self.target_col, axis=1)).iloc[i].values), label-1

  def __len__(self):
    return len(self.image_data)

In [ ]:
train_transform = A.Compose(
    [
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Rotate(limit = 180, p=1, border_mode=cv2.BORDER_REPLICATE),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]
)

val_transform = A.Compose(
    [
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]
)

In [ ]:
dataset = Diabetic_Retinopathy_Dataset()
validation_split = 0.2
shuffle_dataset = True
random_seed = 23

# Creating data indices for training and validation splits:
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

dataset_training = Diabetic_Retinopathy_Dataset(transform = train_transform)
dataset_validation = Diabetic_Retinopathy_Dataset(transform = val_transform)

dataset_training = torch.utils.data.Subset(dataset_training, train_indices)
dataset_validation = torch.utils.data.Subset(dataset_validation, val_indices)

dataloader_training = DataLoader(dataset_training, batch_size=18, shuffle=True, pin_memory= True)
dataloader_validating = DataLoader(dataset_validation, batch_size=18, shuffle=False, pin_memory= True)

# 2. Construct the model and metrics

We use a Resnet34 pretrained and concate them with a CNN model for the tabular data.

In [ ]:
class model(nn.Module):

    def __init__(self, D_in = 8,  D_out=5, device=device):

        super().__init__()

        #image model
        self.mo = models.resnet34(pretrained = True).to(device)
        self.num_ftrs = self.mo.fc.in_features
        self.mo.fc = nn.Linear(self.num_ftrs, 5)
        self.checkpoint = torch.load('C:/Users/48057074t/Desktop/Postgrau/prejecte/checkpoint.pth.tar')
        self.mo.load_state_dict(self.checkpoint['state_dict'])

        # #metadata model
        self.dropout = nn.Dropout(p = 0.25)
        self.relu = nn.ReLU()    

        #Combinació
        self.fc1_comb = nn.Linear(5+D_in, D_out)

    def forward(self, tab, image):

        # image model
        img = self.mo(image)
        img = self.relu(img)

        # Combination
        x = torch.cat((img, tab), dim=1) # this is syntax where we concatenate the 1 channel from image data and 1 from tabular data i.e. 1+1 = 2
        x = self.fc1_comb(x)

        return x

In [ ]:
model = model(D_in = 8, D_out = 5, device = device).to(device)
optimizer = optim.Adam(model.parameters(), lr = learning_rate, weight_decay = 0)
criterion = nn.CrossEntropyLoss(weight = torch.tensor([75782/75782, 75782/4220, 75782/1737, 75782/307, 75782/227]).to(device))
model = model.to(device)


In [ ]:
def adjust_learning_rate(optimizer, epoch, original_lr):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = original_lr * (0.1 ** (epoch // 50))
    # For some models, different parameters are in different groups with different lr
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def correct_predictions(predicted_batch, label_batch):
    pred = predicted_batch.argmax(dim=1, keepdim=True) # get the index of the max log-probability
    
    acum = pred.eq(label_batch.view_as(pred)).sum().item()
    return acum

def save_checkpoint(state, is_best, filename='checkpoint.pth'):
    torch.save(state, filename)
    
    # save an extra copy if it is the best model yet
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')  

In [ ]:
classes = ('1: Normal','2: RD Lleu','3: RD Moderada','4: RD Severa','5: Proliferativa')
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    fig = plt.figure(figsize=(8,8))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

    return fig

# 3. Execution

In [ ]:
def train(train_loader, model, criterion, optimizer, device):

    # Switch to train mode
    model.train()
    loss_mean = []
    acc_mean = []

    for i, (images, input, target) in enumerate(train_loader):
        # Reset gradients
        optimizer.zero_grad()

        #move images to gpu
        images = images.to(device)
        input = input.to(device)
        target = target.to(device)

        # Compute output
        output = model(tab = input, image = images)

        output = output.to(dtype = torch.float, device = device)
        target = target.to(dtype = int, device = device)
        
        loss = criterion(output,target)
        loss.backward()
        optimizer.step()

        acc = 100 * (correct_predictions(output, target) / images.shape[0])

        loss_mean.append(loss.item())
        acc_mean.append(acc)


    return np.mean(acc_mean), np.mean(loss_mean)

In [ ]:
def validate(val_loader, model, criterion, device):

    # Switch to evaluate mode
    model.eval()

    # We will save the values of the accuracies in this list to return the mean of the whole dataset at the end
    acc = 0
    loss = []

    y_pred = []
    y_true = []

    with torch.no_grad():  # We do not need to compute gradients
        for i, (images, input, target) in enumerate(val_loader):

            images = images.to(device)
            input = input.to(device)
            target = target.to(device)

            output = model(image = images, tab = input)

            output = output.to(dtype = torch.float, device = device)
            target = target.to(dtype = int, device = device)

            # loss
            loss.append(criterion(output, target).item())

            # measure accuracy
            acc += correct_predictions(output, target)

            # confusion matrix
            output = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
            y_pred.extend(output) # Save Prediction
            
            target = target.data.cpu().numpy()
            y_true.extend(target) # Save Truth

    cf_matrix = confusion_matrix(y_true, y_pred)
    
    kappa = cohen_kappa_score(y_true, y_pred, weights="quadratic")
    
    f1_micro = f1_score(y_true, y_pred, zero_division=0, average = 'micro')
    
    f1_macro = f1_score(y_true, y_pred, zero_division=0, average = 'macro')

    eval_acc = 100. * acc / len(val_loader.dataset)

    return eval_acc, np.mean(loss), cf_matrix, kappa, f1_micro, f1_macro

In [ ]:
wandb.finish() # This is needed just in case there was a wandb run from a previous execution
wandb.init(project="project_rd")
wandb.run.name = f'run_{num_run}'
best_kappa = -1.
for epoch in range(epochs):
    adjust_learning_rate(optimizer, epoch, learning_rate)

    acc_tr, loss_tr = train(dataloader_training, model, criterion, optimizer, device)

    wandb.log({'Training/accuracy':acc_tr}, epoch)
    wandb.log({'Training/loss':loss_tr}, epoch)

    acc, loss, cf_matrix, kappa, f1_micro, f1_macro = validate(dataloader_validating, model, criterion,device)

    is_best = kappa > best_kappa
    best_kappa = max(kappa, best_kappa)
    
    wandb.log({'Validation/f1_micro': f1_micro},epoch)
    wandb.log({'Validation/f1_macro': f1_macro},epoch)
    wandb.log({'Kappa': kappa},epoch)
    wandb.log({'Validation/accuracy': acc},epoch)
    wandb.log({'Validation/loss':loss},epoch)
    wandb.log({'Conf_matrix':plot_confusion_matrix(cf_matrix, classes,normalize=True)}, epoch) 

    print("Iteracio", epoch,"completada. Validate loss", loss, "train loss", loss_tr, "Validate acc", acc, "train acc", acc_tr, "F1 score micro", f1_micro, "F1 score macro", f1_macro, "Kappa", kappa)

    save_checkpoint({
        'epoch': epoch + 1,
        'arch': "resnet34",
        'state_dict': model.state_dict(),
        'best_kappa': best_kappa,
        'optimizer' : optimizer.state_dict(),
    }, is_best, f"checkpoint_{epoch}.pth.tar")


wandb.finish()
